In [1]:
import sys
sys.path.append('../')
from utils import preprocess

import numpy as np
import h5py

import pandas as pd

from collections import defaultdict
import string

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, KFold, GridSearchCV

import keras
from keras.layers import Dense, GlobalAveragePooling1D, Embedding, Lambda
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

np.random.seed(1234)

Using TensorFlow backend.


In [2]:
def create_docs(df, n_gram_max=1):
    docs = []

    for i, text in enumerate(df.text):    
        def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(2, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return ngrams + q

        doc = preprocess(text).split()
                        
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
        
    return docs

In [3]:
df = pd.read_csv('./../data/train_feature.csv')
df_test = pd.read_csv('./../data/test_feature.csv')
text = df.text.values
text_test = df_test.text.values

author2class = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
class2author = ['EAP', 'HPL', 'MWS']
y = np.array([author2class[a] for a in df.author])
y = to_categorical(y)

In [4]:
def create_model(input_dim, embeddings_dims=10):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [5]:
n_gram_max = 2
embedding_dims = 10
maxlen = 1024

raw_docs = create_docs(df, n_gram_max=n_gram_max)
raw_docs_test = create_docs(df_test, n_gram_max=n_gram_max)

seed = 7
num_split = 5
epochs = 60

# for next training
predict_prob_features = np.zeros((len(df), 3))
predict_prob_features_test = np.zeros((len(df_test), 3))

ite = 0
sum_loss = 0.
min_count = 2

kf = KFold(n_splits=num_split, random_state=seed, shuffle=True)
for train_index, val_index in kf.split(text):
    ite += 1
    print('{}/{}: #Trains: {}, #Val: {}'.format(ite, num_split, len(train_index), len(val_index)), end=' ')
    
    docs_train = [raw_docs[i] for i in train_index]
    docs_val = [raw_docs[i] for i in val_index]

    # get vocab
    tokenizer = Tokenizer(filters='', lower=True)
    #     tokenizer.fit_on_texts(docs_train)
    tokenizer.fit_on_texts(docs_train + docs_val)

    num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])
    
    tokenizer = Tokenizer(num_words=num_words, filters='', lower=True)
    tokenizer.fit_on_texts(docs_train + docs_val)    

    docs_train = tokenizer.texts_to_sequences(docs_train)
    docs_val   = tokenizer.texts_to_sequences(docs_val)
    docs_test  = tokenizer.texts_to_sequences(raw_docs_test)

    x_train = pad_sequences(sequences=docs_train, maxlen=maxlen)
    x_val  = pad_sequences(sequences=docs_val, maxlen=maxlen)
    x_test = pad_sequences(sequences=docs_test, maxlen=maxlen)

    y_train, y_val = y[train_index], y[val_index]

    input_dim = max(np.max(x_train), np.max(x_val)) + 1
    print('#vocab: {} '.format(num_words), end=' ')
    print(x_train.shape, x_val.shape, x_test.shape)

    model = create_model(input_dim)
    
    checkpointer = ModelCheckpoint(filepath='./../fasttext_weights/weights.hdf5', verbose=0, save_best_only=True)

    hist = model.fit(x_train, y_train,
                     batch_size=16,
                     validation_data=(x_val, y_val),
                     epochs=epochs,
                     callbacks=[EarlyStopping(patience=4, monitor='val_loss'), 
                                checkpointer])

    # load best weights
    model.load_weights('./../fasttext_weights/weights.hdf5')
    y_pred = model.predict_proba(x_val)
    sum_loss += log_loss(y_pred=y_pred, y_true=np.nonzero(y_val)[1])
    
    # save features
    predict_prob_features[val_index] = y_pred
    predict_prob_features_test += model.predict_proba(x_test)
    print('valLoss: {}'.format(sum_loss/ite))

1/5: #Trains: 15663, #Val: 3916 #vocab: 75122  (15663, 1024) (3916, 1024) (8392, 1024)
Train on 15663 samples, validate on 3916 samples
Epoch 1/60
15663/15663 [==============================] - 20s - loss: 1.0861 - acc: 0.4037 - val_loss: 1.0826 - val_acc: 0.4032
Epoch 2/60
15663/15663 [==============================] - 19s - loss: 1.0759 - acc: 0.4037 - val_loss: 1.0722 - val_acc: 0.4040
Epoch 3/60
15663/15663 [==============================] - 20s - loss: 1.0533 - acc: 0.4121 - val_loss: 1.0421 - val_acc: 0.4096
Epoch 4/60
15663/15663 [==============================] - 20s - loss: 1.0114 - acc: 0.4753 - val_loss: 1.0002 - val_acc: 0.4517
Epoch 5/60
15663/15663 [==============================] - 20s - loss: 0.9538 - acc: 0.5961 - val_loss: 0.9507 - val_acc: 0.5008
Epoch 6/60
15663/15663 [==============================] - 20s - loss: 0.8879 - acc: 0.6930 - val_loss: 0.8910 - val_acc: 0.6241
Epoch 7/60
15663/15663 [==============================] - 22s - loss: 0.8200 - acc: 0.7499 - val

15663/15663 [==============================] - 21s - loss: 0.6784 - acc: 0.8110 - val_loss: 0.7147 - val_acc: 0.7651
Epoch 10/60
15663/15663 [==============================] - 20s - loss: 0.6231 - acc: 0.8291 - val_loss: 0.6704 - val_acc: 0.7845
Epoch 11/60
15663/15663 [==============================] - 20s - loss: 0.5734 - acc: 0.8437 - val_loss: 0.6359 - val_acc: 0.7817
Epoch 12/60
15663/15663 [==============================] - 19s - loss: 0.5286 - acc: 0.8579 - val_loss: 0.6034 - val_acc: 0.7998
Epoch 13/60
15663/15663 [==============================] - 19s - loss: 0.4894 - acc: 0.8667 - val_loss: 0.5803 - val_acc: 0.7960
Epoch 14/60
15663/15663 [==============================] - 19s - loss: 0.4520 - acc: 0.8801 - val_loss: 0.5527 - val_acc: 0.7980
Epoch 15/60
15663/15663 [==============================] - 19s - loss: 0.4192 - acc: 0.8881 - val_loss: 0.5325 - val_acc: 0.8113
Epoch 16/60
15663/15663 [==============================] - 20s - loss: 0.3892 - acc: 0.8966 - val_loss: 0.505

15663/15663 [==============================] - 19s - loss: 0.3253 - acc: 0.9185 - val_loss: 0.4656 - val_acc: 0.8386
Epoch 20/60
15663/15663 [==============================] - 19s - loss: 0.3028 - acc: 0.9252 - val_loss: 0.4532 - val_acc: 0.8424
Epoch 21/60
15663/15663 [==============================] - 19s - loss: 0.2824 - acc: 0.9309 - val_loss: 0.4399 - val_acc: 0.8473
Epoch 22/60
15663/15663 [==============================] - 19s - loss: 0.2633 - acc: 0.9375 - val_loss: 0.4354 - val_acc: 0.8378
Epoch 23/60
15663/15663 [==============================] - 19s - loss: 0.2455 - acc: 0.9431 - val_loss: 0.4173 - val_acc: 0.8524
Epoch 24/60
15663/15663 [==============================] - 20s - loss: 0.2291 - acc: 0.9484 - val_loss: 0.4097 - val_acc: 0.8483
Epoch 25/60
15663/15663 [==============================] - 20s - loss: 0.2139 - acc: 0.9500 - val_loss: 0.3995 - val_acc: 0.8560
Epoch 26/60
15663/15663 [==============================] - 20s - loss: 0.1994 - acc: 0.9563 - val_loss: 0.392

15663/15663 [==============================] - 19s - loss: 0.1291 - acc: 0.9740 - val_loss: 0.3746 - val_acc: 0.8560
Epoch 32/60
15663/15663 [==============================] - 20s - loss: 0.1211 - acc: 0.9753 - val_loss: 0.3615 - val_acc: 0.8611
Epoch 33/60
15663/15663 [==============================] - 21s - loss: 0.1127 - acc: 0.9778 - val_loss: 0.3623 - val_acc: 0.8618
Epoch 34/60
15663/15663 [==============================] - 21s - loss: 0.1054 - acc: 0.9791 - val_loss: 0.3736 - val_acc: 0.8481
Epoch 35/60
15663/15663 [==============================] - 20s - loss: 0.0980 - acc: 0.9809 - val_loss: 0.3533 - val_acc: 0.8647
Epoch 36/60
15663/15663 [==============================] - 20s - loss: 0.0914 - acc: 0.9840 - val_loss: 0.3493 - val_acc: 0.8657
Epoch 37/60
15663/15663 [==============================] - 20s - loss: 0.0861 - acc: 0.9850 - val_loss: 0.3492 - val_acc: 0.8654
Epoch 38/60
15663/15663 [==============================] - 20s - loss: 0.0792 - acc: 0.9862 - val_loss: 0.345

7968/8392 [===========================>..] - ETA: 0svalLoss: 0.33823418763046964


In [6]:
name = 'fasttext_bigram_reverse'
for a, c in author2class.items():
    df['{}_{}'.format(a, name)] = predict_prob_features[:, c]
    df_test['{}_{}'.format(a, name)] = predict_prob_features_test[:, c]/num_split

In [7]:
df.to_csv('./../data/train_feature.csv')
df_test.to_csv('./../data/test_feature.csv')